# Visualising the IRX cluster

## finding IRX gene candidates

We compiled a list of homeobox genes in the IRX family from the [Aase-Remedios _et al._
(2023)](https://doi.org/10.1093/molbev/msad239) analysis of the spider Homeobox gene repertoire. We
used this list to scan against the _P. litorale_ draft genome using `mmseqs`:

```bash
cd /lisc/scratch/zoology/pycnogonum/genome/draft/irx
module load mmseqs2
M8FORMAT="query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qlen"
mmseqs easy-search irx_chelicerates.fa ../draft.fasta irx_genomic.m8 tmp --format-output $M8FORMAT --threads 4
```

First, we'll have to read the GFF file so that we can locate the loci that overlap with our genomic
hits.

In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs

from matplotlib import pyplot as plt
from matplotlib.patches import ConnectionPatch

In [2]:
def read_gff(loc):
    gff = pd.read_csv(loc, sep="\t", header=None, skiprows=4)
    gff_columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    gff.columns = gff_columns
    return gff

# read file, name columns
def read_aln(m8, id_sep=None):
    irx = pd.read_csv(m8, sep="\t", header=None)
    m8_columns = ['query', 'target', 'seq_id', 'ali_len', 'no_mism', 'no_go',
                'q_start', 'q_end', 't_start', 't_end', 'eval', 'bit', 'qlen']
    irx.columns = m8_columns
    # trim the query to just the ID
    if id_sep is not None:
        irx["query"] = irx["query"].str.split(id_sep).str[1]
    return irx

In [3]:
gff_loc = "/Volumes/project/pycnogonum/paper/zenodo/results/merged_sorted_named_dedup_flagged.gff3"
tmp = read_gff(gff_loc)
gff = tmp[tmp['type'] == 'gene'].copy()
gff['gene'] = gff['attributes'].str.split("ID=").str[1].str.split(";").str[0]
del tmp

In [4]:
irx = read_aln('/Volumes/scratch/pycnogonum/genome/draft/irx/irx_genomic.m8')

In [5]:
irx['species'] = irx['query'].str.split("_").str[:2].str.join(" ")
irx['symbol'] = irx['query'].str.split("_").str[2:].str.join("_")

In [6]:
def hit_to_locus(hit, gff):
    same_cirxm = gff['seqid'] == hit['target']
    within_borders = (gff['start'] <= hit['t_start']) & (gff['end'] >= hit['t_end'])

    overlap = gff[same_cirxm & within_borders]
    return overlap['gene'].values[0]

In [7]:
hits = irx[irx['eval'] < 1e-20].copy()
hits['gene'] = hits.apply(hit_to_locus, axis=1, gff=gff)

In [8]:
candidates = hits['gene'].unique()

In [9]:
plit_peptides = {}
with open('/Volumes/project/pycnogonum/genome/draft/transcripts.fa.transdecoder.pep', 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        if line.startswith(">"):
            gene = line.split(" ")[0][1:]
            plit_peptides[gene] = ""
        else:
            plit_peptides[gene] += line.strip()

In [10]:
candidate_sequences = {}
for gene in plit_peptides:
    for c in candidates:
        if c in gene:
            candidate_sequences[gene] = plit_peptides[gene]

In [11]:
with open('/Volumes/scratch/pycnogonum/genome/draft/irx/irx_candidates.fa', 'w') as f:
    for gene, seq in candidate_sequences.items():
        f.write(f">{gene}\n{seq}\n")

Now we can concatenate all the sequences and align them.

```bash
cat irx_chelicerates.fa > irx.fa
cat irx_candidates.fa >> irx.fa
```

Before continuing, check whether the sequences were concatenated correctly - if there is no newline
character at the end of `irx.fa`, the header of the first candidate sequence will be appended to
the last sequence of the `irx_chelicerates.fa` file.

Additionally, we need to check for synonyms here; where I found the same combination of species and
gene name, I collapsed identical sequences. Where this was not given, I appended lowercase "i"s to
the names in order to keep the additional variation.

First, quick-and-dirty alignment so we can just cut out the homeodomain regions from the _P.
litorale_ sequences:

```bash
mafft --thread 4 --maxiterate 1000 --localpair irx.fa > irx_aligned.fasta
```

The resulting alignment was input to IQTREE2.

```bash
module load iqtree
/usr/bin/time iqtree2 -s ./irx_aligned.fasta -B 1000 -T 4
```

In [12]:
names = {
    'PB.1135': 'Irx2/mirr',
    'PB.1137': 'Irx3',
    'g10269': 'Irx1/ara'
}

irx_cluster = gff.set_index('gene').loc[names.keys()]
irx_cluster['name'] = irx_cluster.index.map(names)
irx_cluster.sort_values(['seqid', 'start'], ascending=[False, True])

,seqid,source,type,start,end,score,strand,phase,attributes,name
gene,,,,,,,,,,
g10269,pseudochrom_7,AUGUSTUS,gene,2015173,2029196,.,+,.,ID=g10269;function=Motif in Iroquois-class hom...,Irx1/ara
PB.1135,pseudochrom_7,PacBio,gene,2089725,2104329,.,+,.,ID=PB.1135;function=Motif in Iroquois-class ho...,Irx2/mirr
PB.1137,pseudochrom_7,PacBio,gene,2377455,2424480,.,+,.,ID=PB.1137;function=establishment of animal or...,Irx3
